In [1]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

if not solver:
    print('Solver not created.')
    exit()

distance_matrix = [
    [0, 20, 25, 35, 65, 90, 85, 80, 86, 25, 35, 20, 44, 35, 82],  # Heathrow
    [20, 0, 15, 35, 60, 55, 57, 85, 90, 25, 35, 30, 37, 20, 40],  # Harrow
    [25, 15, 0, 30, 50, 70, 55, 50, 65, 10, 25, 15, 24, 20, 90],  # Ealing
    [35, 35, 30, 0, 45, 60, 53, 55, 47, 12, 22, 20, 12, 10, 21],  # Holborn
    [65, 60, 50, 45, 0, 46, 15, 45, 75, 25, 11, 19, 15, 25, 25],  # Sutton
    [90, 55, 70, 60, 46, 0, 15, 25, 45, 65, 53, 43, 63, 70, 27],  # Dartford
    [85, 57, 55, 53, 15, 15, 0, 17, 25, 41, 25, 33, 27, 45, 30],  # Bromley
    [80, 85, 50, 55, 45, 25, 17, 0, 25, 40, 34, 32, 20, 30, 10],  # Greenwich
    [86, 90, 65, 47, 75, 45, 25, 25, 0, 65, 70, 72, 61, 45, 13],  # Barking
    [25, 25, 10, 12, 25, 65, 41, 40, 65, 0, 20, 8, 7, 15, 25],    # Hammersmith
    [35, 35, 25, 22, 11, 53, 25, 34, 70, 20, 0, 5, 12, 45, 65],   # Kingston
    [20, 30, 15, 20, 19, 43, 33, 32, 72, 8, 5, 0, 14, 34, 56],    # Richmond
    [44, 37, 24, 12, 15, 63, 27, 20, 61, 7, 12, 14, 0, 30, 40],   # Battersea
    [35, 20, 20, 10, 25, 70, 45, 30, 45, 15, 45, 34, 30, 0, 27],  # Islington
    [82, 40, 90, 21, 25, 27, 30, 10, 13, 25, 65, 56, 40, 27, 0]   # Woolwich
]

num_cities = len(distance_matrix)
num_vehicles = 6

used = [solver.BoolVar(f'vehicle_{i}_used') for i in range(num_vehicles)]
path = [[[solver.BoolVar(f'vehicle_{k}_from_{i}_to_{j}') for j in range(num_cities)] for i in range(num_cities)] for k in range(num_vehicles)]
visit = [[solver.BoolVar(f'vehicle_{k}_visits_city_{i}') for i in range(num_cities)] for k in range(num_vehicles)]
u = [[solver.IntVar(0,num_cities, f'position_of_{i}_in_vehicle_{k}') for i in range(num_cities)] for k in range(num_vehicles)]

for k in range(num_vehicles):
    for i in range(num_cities):
        solver.Add(path[k][i][i] == 0)  
        solver.Add(visit[k][i] <= used[k])  
        solver.Add(sum(path[k][j][i] for j in range(num_cities)) == visit[k][i])
        solver.Add(sum(path[k][i][j] for j in range(num_cities)) == visit[k][i])  

    solver.Add(sum(distance_matrix[i][j] * path[k][i][j] for i in range(num_cities) for j in range(1,num_cities)) <= 120)

for i in range(num_cities):
    if i == 0:
        for k in range(num_vehicles):
            solver.Add(visit[k][i] == used[k])
    else:
        solver.Add(sum(visit[k][i] for k in range(num_vehicles)) == 1)

for k in range(num_vehicles):
    solver.Add(sum(path[k][0][j] for j in range(1, num_cities)) == used[k])
    solver.Add(sum(path[k][j][0] for j in range(1, num_cities)) == used[k])

for k in range(num_vehicles):
    for i in range(1, num_cities):
        for j in range(1, num_cities):
            if i != j:
                solver.Add(u[k][i] - u[k][j] + num_cities * path[k][i][j] <= num_cities - 1)

for k in range(num_vehicles):
    for i in range(num_cities):
        solver.Add(u[k][i] >= visit[k][i])
        solver.Add(u[k][i] <= num_cities * visit[k][i])
for i in range(6):
    for j in range(6):
        if i<=j:
            solver.Add(sum(visit[i][k] for k in range(num_cities))>=sum(visit[j][k] for k in range(num_cities)))

solver.Minimize(sum(used[k] for k in range(num_vehicles)))

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    for k in range(num_vehicles):
        if used[k].solution_value() == 1:
            print(f'Route for vehicle {k}:')
            route = []
            current_city = 0
            while True:
                route.append(current_city+1)
                next_city = None
                for j in range(num_cities):
                    if path[k][current_city][j].solution_value() > 0.5:
                        next_city = j
                        break
                if next_city is None or next_city == 0:
                    break
                current_city = next_city
            print(route)
            total_distance = sum(distance_matrix[i][j] * path[k][i][j].solution_value() for i in range(num_cities) for j in range(1,num_cities))
            #print(f'Total distance: {total_distance}')
else:
    print('The problem does not have an optimal solution.')

Solution:
Objective value = 2.0
Route for vehicle 0:
[1, 2, 3, 10, 4, 13, 7, 6]
Route for vehicle 1:
[1, 12, 11, 5, 14, 8, 15, 9]


In [2]:
import pandas as pd 
df=pd.read_excel("performance/DMU.xlsx")
print(df)

       DMU  Total Cost [103$/yr]  Staff Cost [103$/yr]  \
0   DMU-01                  1310                   238   
1   DMU-02                  2091                   459   
2   DMU-03                   930                   154   
3   DMU-04                  3591                   795   
4   DMU-05                  2729                   571   
5   DMU-06                  2030                   497   
6   DMU-07                  3247                   558   
7   DMU-08                  2501                   571   
8   DMU-09                  2299                   407   
9   DMU-10                  2413                   306   
10  DMU-11                  1450                   458   
11  DMU-12                  2758                   494   
12  DMU-13                  1857                   360   
13  DMU-14                  3195                   618   
14  DMU-15                  3505                   759   
15  DMU-16                  1408                   313   
16  DMU-17    

In [1]:
!pip install pandas openpyxl


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\olw09\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
import pandas as pd

# Data for the Excel sheet
data = {
    't − 0': [111.0, 82.5, 70.0, 154.6, 110.8],
    't − 1': [108.1, 81.6, 73.7, 152.4, 108.0],
    't − 2': [107.9, 80.1, 72.3, 146.1, 103.7],
    't − 3': [108.5, 83.1, 69.7, 157.5, 106.6],
    't − 4': [111.4, 85.0, 69.5, 168.4, 107.3],
    't − 5': [115.5, 92.6, 74.8, 166.9, 109.5],
    't − 6': [113.2, 91.6, 73.8, 164.1, 108.7],
    't − 7': [111.9, 88.3, 70.2, 169.0, 111.1],
    't − 8': [99.7, 80.8, 64.3, 143.8, 101.0],
    't − 9': [105.1, 86.1, 71.8, 151.3, 105.4],
    't − 10': [100.9, 81.8, 71.7, 148.3, 109.6],
    't − 11': [105.0, 85.6, 69.5, 140.6, 112.8],
    't − 12': [105.2, 84.6, 70.5, 131.5, 113.6],
    't − 13': [107.0, 90.3, 74.9, 138.0, 117.4],
    't − 14': [109.0, 88.3, 78.5, 135.4, 123.1],
    't − 15': [111.4, 85.6, 73.0, 114.0, 124.5],
    't − 16': [107.2, 81.6, 74.5, 116.1, 118.7],
    't − 17': [111.3, 87.4, 75.0, 121.6, 125.0],
    't − 18': [108.6, 87.5, 77.0, 127.6, 127.7],
    't − 19': [105.6, 86.6, 72.4, 116.2, 121.2],
    't − 20': [105.9, 90.0, 71.4, 128.6, 125.1],
    't − 21': [104.7, 91.4, 68.9, 129.4, 119.9],
    't − 22': [107.7, 95.3, 69.7, 137.1, 117.9],
    't − 23': [107.4, 92.9, 68.6, 134.5, 124.6],
    't − 24': [108.0, 97.0, 70.2, 156.0, 124.3],
    't − 25': [104.7, 102.6, 74.3, 159.5, 128.8],
    't − 26': [112.8, 107.0, 76.3, 155.9, 134.2],
    't − 27': [109.7, 110.4, 86.0, 155.0, 140.9],
    't − 28': [111.7, 109.7, 88.9, 149.9, 138.2],
    't − 29': [120.4, 105.9, 83.5, 153.5, 141.6],
    't − 30': [118.0, 105.9, 83.4, 153.0, 140.6],
    't − 31': [119.7, 103.0, 84.9, 149.9, 158.2],
    't − 32': [116.7, 102.4, 84.9, 153.7, 149.6],
    't − 33': [115.8, 107.2, 86.1, 167.0, 152.8],
    't − 34': [113.7, 104.5, 78.9, 181.0, 144.3],
    't − 35': [115.7, 105.8, 79.5, 189.4, 155.5],
    't − 36': [114.4, 104.8, 79.1, 197.9, 154.2],
    't − 37': [113.8, 103.8, 79.9, 201.7, 154.5],
    't − 38': [114.0, 107.0, 82.0, 196.3, 158.0],
    't − 39': [114.1, 107.4, 77.2, 188.0, 163.8],
    't − 40': [111.5, 112.0, 78.5, 189.9, 164.3],
    't − 41': [109.2, 106.8, 77.1, 172.1, 163.9],
    't − 42': [110.1, 105.3, 76.1, 178.0, 165.6],
    't − 43': [112.8, 113.1, 82.6, 171.0, 161.4],
    't − 44': [111.0, 116.6, 91.4, 179.5, 165.4],
    't − 45': [105.6, 126.7, 94.5, 180.6, 160.9],
    't − 46': [107.3, 123.1, 90.0, 173.5, 162.0],
    't − 47': [103.2, 112.3, 88.5, 164.4, 153.0],
    't − 48': [102.8, 103.1, 81.8, 164.2, 141.2],
    't − 49': [93.9, 95.0, 80.7, 153.0, 133.4],
    't − 50': [93.6, 92.7, 80.5, 164.0, 139.3]
}

# Creating DataFrame
df = pd.DataFrame(data, index=[ 'RD', 'AKZ', 'KLM', 'PHI', 'UN'])

# Saving to Excel file
df.to_excel('stock_values.xlsx', engine='openpyxl')

print("Excel file 'stock_values.xlsx' has been created.")


Excel file 'stock_values.xlsx' has been created.


In [19]:
df = pd.read_excel('stock_values.xlsx')
print(df['items'])
# Printing row for 'RD'
df1=df = pd.read_excel('stock_values.xlsx', index_col=0)


0     RD
1    AKZ
2    KLM
3    PHI
4     UN
Name: items, dtype: object
